In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import glob
import os
import numpy as np
import pandas as pd
import sys
import cv2

In [2]:
class FaceImage(Dataset):
    def __init__(self,label,picture_file,path,Train=True):
        self.data_path = path
        self.label = label
        self.train = Train
        self.picture_file = picture_file
    def __getitem__(self,index):

        file_path = os.path.join(self.data_path,self.picture_file[index])
        img = cv2.imread(file_path,cv2.IMREAD_GRAYSCALE)
        img = np.expand_dims(img,0)
        img =torch.tensor(img).float()
        if self.train==True:
            return torch.FloatTensor(img),self.label[index,1]
        else:
            return img

    def __len__(self):
        return len(os.listdir(self.data_path))


In [3]:
class ImageNet(nn.Module):
    def __init__(self):
        super(ImageNet,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 48, kernel_size=3,stride =(1,1)),
            nn.Dropout2d(0.3),
            nn.MaxPool2d(2),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(48, 128, kernel_size=3,stride =(1,1)),
            nn.Dropout2d(0.2),
            nn.MaxPool2d(2),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3,stride =(1,1)),
            nn.Dropout2d(0.5),
            nn.MaxPool2d(2),
            nn.ReLU()
        )

        self.fcn1 = nn.Linear(in_features =4096 ,out_features =512 ,bias = True)
        self.fcn2 = nn.Linear(in_features = 512,out_features =256 ,bias = True)
        self.fcn3 = nn.Linear(in_features =256 ,out_features =7 ,bias = True)

    def forward(self , x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0),-1)
        #print("size:",x.shape)
        x = F.relu(self.fcn1(x))
        x = F.relu(self.fcn2(x))
        out = self.fcn3(x)
        
        return out

In [4]:
train_label = pd.read_csv('./train.csv')
train_label = np.array(train_label,dtype = int)
datalen  = len(train_label)
picture_file = sorted(os.listdir('./train_img'))
train_dataset = FaceImage(train_label,picture_file,'./train_img')
picture_file = sorted(os.listdir('./test_img'))
test_dataset = FaceImage(train_label,picture_file,'./test_img',Train = False)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = 256)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size =256)

In [8]:
model = torch.load("Imagenet_best.pkl",map_location = "cpu")
model = model.cpu()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'torch.nn.modules.pooling.MaxPool2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'torch.nn.modules.ac

In [9]:
model.eval()
ans = []
for idx,test in enumerate(_loader):
    test =test
    #print(idx)
    ans.append(model(test))


In [10]:
ans_final =[]
for a in ans:
    for pre in a:
        pre = pre.tolist()
        ans_final.append(pre.index(max(pre)))

3


In [11]:
with open("reconstru_result.csv","w") as f:
    print("id,label", file = f)
    for id,label in enumerate(ans_final):
        print("{},{}".format(id,label) ,file = f)